In [1]:
import bs4
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


프로젝트에 필요한 라이브러리들을 임포트합니다. Beautiful Soup(bs4)를 웹 스크래핑에, LangChain 관련 라이브러리들을 LLM 연동 및 문서 처리에, OpenAI API를 사용하기 위한 모듈을 불러옵니다.
# LLM 설정 및 API 키 확인

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(f"API key exists: {api_key is not None}")

API key exists: True


OpenAI의 gpt-4o-mini 모델을 사용하기 위한 초기 설정을 합니다. .env 파일에서 환경변수를 불러와 API 키를 확인하고, 키가 존재하는지 확인합니다.

# 웹페이지 불러오기

In [ ]:
loader = WebBaseLoader(
  web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
  bs_kwargs=dict(
    parse_only=bs4.SoupStrainer(
            class_=("css-j3idia", "editedContent")
        )
  ),
)

docs = loader.load()
print(f"로드된 문서 수: {len(docs)}")
print(f"첫 번째 문서 내용 일부: {docs[0].page_content[:100] if docs else 'No content'}")


로드된 문서 수: 1
첫 번째 문서 내용 일부: 'AII-in 코딩 공모전’ 수상작을 소개합니다코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩


LangChain의 WebBaseLoader를 사용하여 스파르타코딩클럽의 ALL-in 코딩 공모전 수상작 페이지를 불러옵니다.
Beautiful Soup의 SoupStrainer를 활용해 문서의 내용이 들어있는 ("css-j3idia", "editedContent")부분을 파싱하여 필요한 콘텐츠만 추출합니다.
로드된 문서의 수와 첫 문서의 일부분을 출력하여 정상적으로 데이터가 불러와졌는지 확인합니다.

# 문서 청크로 나누기



In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10
)

splits = text_splitter.split_documents(docs)
print(f"분할된 청크 수: {len(splits)}")

if len(splits) == 0:
    print("청크가 없어 원본 문서를 사용합니다.")
    splits = docs

분할된 청크 수: 12


불러온 문서를 처리하기 쉽도록 청크(chunks)로 나눕니다. RecursiveCharacterTextSplitter를 사용하여 문서를 500자 단위로 분할하고, 청크 간 10자의 중복을 허용합니다. 분할된 청크의 수를 출력하고, 만약 청크가 생성되지 않았다면 원본 문서를 그대로 사용합니다.

# 벡터 스토어 생성 및 리트리버 설정

In [56]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
)
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 50}
)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

분할된 문서들을 벡터화하고, Chroma 벡터 데이터베이스에 저장합니다.
이후 이 벡터 스토어에서 문서를 검색할 수 있는 리트리버(retriever)를 생성합니다.
기본 검색 방식은 "similarity"(유사도 기반)이며, 최대 50개의 관련 문서를 반환하도록 설정합니다.

# 사용자 질문 및 문서 검색

In [ ]:
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

print(len(retrieved_docs))
retrieved_docs

사용자 질문을 정의하고, 해당 질문에 관련된 문서를 리트리버를 통해 검색합니다. 검색된 문서의 수를 출력하고, 검색된 문서 목록을 확인합니다.

# 프롬프트 생성

In [58]:
prompt = ChatPromptTemplate.from_template("""너는 이 공모전을 평가했던 심사위원이이야.한국어로 대답하고,
  다음에 이 공모전을 참가할 사람들이 참고할 수 있도록 유익한 정보들을 보기 쉽게 정리해줄 수 있어야해.
  정보를 찾은 후 대답을 정리할 때는 너의 의견이 들어가면 안돼. 리트리버에서 가져온 정보는 가능한 모두 활용해.
  그리고 내가 제공한 문서에서 질문에 대한 대답을 찾을 수 없다면, 모른다고 대답해야해. <context>: {context} <question>: {question}""")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='너는 이 공모전을 평가했던 심사위원이이야.한국어로 대답하고,\n  다음에 이 공모전을 참가할 사람들이 참고할 수 있도록 유익한 정보들을 보기 쉽게 정리해줄 수 있어야해.\n  정보를 찾은 후 대답을 정리할 때는 너의 의견이 들어가면 안돼. 리트리버에서 가져온 정보는 가능한 모두 활용해.\n  그리고 내가 제공한 문서에서 질문에 대한 대답을 찾을 수 없다면, 모른다고 대답해야해. <context>: {context} <question>: {question}'), additional_kwargs={})]


LLM에게 전달할 프롬프트 템플릿을 생성합니다. 프롬프트는 공모전 심사위원 역할을 부여하고, 검색된 문서의 정보만을 활용하여 객관적인 정보를 제공하도록 지시합니다.

# LLM 응답 요청
검색된 문서와 사용자 질문을 프롬프트에 넣고, LLM에 응답을 요청한 후 결과를 출력합니다.

In [ ]:
response = llm.invoke(user_prompt)
print(response.content)

## AII-in 코딩 공모전 수상작 요약

### 🏆 대상 수상작
- **[Lexi Note]**
  - **제작자**: 다나와(김다애, 박나경)
  - **소개**: 어문학 전공생을 위한 언어 공부 필기 웹 서비스. 단어를 드래그하면 네이버 사전과 연동되어 의미를 찾고 필기를 동시에 할 수 있으며, 긴 문장은 번역기와 연결하여 쉽게 이해할 수 있다. 할일 목록과 스케줄 템플릿 제공.

### 🎖️ 우수상
- **[에코 클래스룸]**
  - **제작자**: This is 스파게티!!!(박지성, 김서원, 박범수)
  - **소개**: 수업 실시간 소통 서비스로, 학생들이 익명으로 의견이나 질문을 제출할 수 있게 하여 교수 매칭 피드백을 받을 수 있는 서비스. 학생의 이해도를 테스트하는 퀴즈 생성 기능도 포함되어 있다.
  
- **[우리집 히어로즈]**
  - **제작자**: 인트(배정연, 한지수)
  - **소개**: 자취방에서 발생하는 벌레 문제를 해결하기 위한 매칭 서비스. 사용자가 벌레 퇴치 요청을 올리면, 학교 내 벌레 퇴치 히어로와 매칭되고, 사용자 신원이 보장된 안전한 환경에서 수행된다.

### 🏅 입선작
- **[학교생활 매니저]**
  - **제작자**: 아이칼F4(조민제, 이민기, 강건, 박근우)
  - **소개**: 학교 생활을 효율적으로 관리할 수 있는 앱으로, 일정과 과제 관리, 성적 예측, 학점 계산 등 다양한 기능을 제공한다. 캘린더와 공지사항 기능으로 중요한 정보를 놓치지 않게 돕는다.

- **[BLOTIE]**
  - **제작자**: 블로티(이은주, 한명수, 황준영)
  - **소개**: 교내 외국인과 내국인을 연결하는 매칭 플랫폼. 학생 간의 문화와 언어 교류를 도와주며, 실시간 채팅과 피드 기능으로 자유로운 소통을 지원한다.

- **[Crewing]**
  - **제작자**: 동학대학운동(김민아, 임경진, 신은혜, 고수)
  - **소개**: 대학생들이 연합 동아리에 쉽고 효율적으로 가입할 수 있도록 지원하는 플랫폼. 회

# 관련없는 질문 테스트
문서와 관련 없는 질문을 했을 때 모델이 문서에 기반한 답변만 제공하는지 테스트합니다.

In [ ]:
unrelated_msg = "6대 차류에 대해 설명해줘."
user_prompt2 = prompt.invoke({"context": format_docs(retrieved_docs), "question": unrelated_msg})

response = llm.invoke(user_prompt2)
print(response.content)

제공된 문서에는 'All-in 코딩 공모전'의 수상작에 대해 구체적인 설명이 있지만, 수상작이 총 몇 개이며 그 목록이나 상세한 내용은 포함되어 있지 않습니다. 따라서 6대 차류에 대한 정보를 제공할 수 없습니다.


# 문제점
리트리버에서 관련된 정보를 전부 가져오지 않아 답변에서도 수상작 한 개만 가져오는 문제가 있었습니다.

## 리트리버 수정
리트리버가 반환하는 문서의 최대 개수를 지정하여 문서를 여러개 가져올 수 있도록 했습니다.
1. similarity 리트리버

In [ ]:
retriever_similarity = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 50})
retrieved_docs_similarity = retriever.invoke(user_msg)

print(retrieved_docs_similarity)

2. mmr 리트리버
MMR은 관련성과 다양성을 모두 고려하여 검색 결과의 중복을 줄이고 다양한 정보를 제공하는 데 유용합니다.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

retriever_mmr = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 50})
retrieved_docs_mmr = retriever.invoke(user_msg)

print(retrieved_docs_mmr)

3. 문서압축 리트리버
LLM을 이용해 검색된 문서를 압축하여 질문과 관련된 핵심 내용만 추출합니다.

In [ ]:
base_retriever_context = vectorstore.as_retriever(
                                search_type='similarity',
                                search_kwargs={'k':50})
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever_context
)

compressed_docs_context = compression_retriever.get_relevant_documents(user_msg)
print(len(compressed_docs_context))
compressed_docs_context

4. self-query 리트리버
이 방법은 메타데이터 필드를 정의하여 LLM이 사용자 질문을 구조화된 쿼리로 변환하도록 합니다. 공모전과 관련된 메타데이터(상장 종류, 공모전 이름, 수상자 등)를 정의하여 보다 정확한 검색이 가능하도록 합니다.

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# 수상 정보 정의
metadata_field_info = [
    AttributeInfo(
        name="award_type",
        description="상장 종류 (대상, 최우수상, 우수상, 입선선 등)",
        type="string",
    ),
    AttributeInfo(
        name="competition_name", 
        description="공모전 이름", 
        type="string"
    ),
    AttributeInfo(
        name="recipients_name", 
        description="수상자들의의 이름", 
        type="string"
    ),
    AttributeInfo(
        name="recipient_work_decription", 
        description="수상작에 대한 설명", 
        type="string"
    ),
    AttributeInfo(
        name="tech_stack", 
        description="사용한 기술", 
        type="string"
    ),
]

self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_contents="ALL-in 코딩 공모전 수상작 정보",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 50},  # k 의 값을 2로 지정하여 검색 결과를 2개로 제한합니다.
)

retrieved_docs_self_query = self_query_retriever.get_relevant_documents(user_msg)
print(len(retrieved_docs_self_query))
retrieved_docs_self_query


리트리버들의 결과 비교

In [66]:
print("1. search type: similarity")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_similarity), "question": user_msg})
response = llm.invoke(user_prompt)
print(response.content)


print("\n\n\n2. search type: mmr")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_mmr), "question": user_msg})
response = llm.invoke(user_prompt)


print(response.content)
print("\n\n\n3. contextual compression retriever")
user_prompt = prompt.invoke({"context": format_docs(compressed_docs_context), "question": user_msg})
response = llm.invoke(user_prompt)


print(response.content)
print("\n\n\n4. self-query retriever")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_self_query), "question": user_msg})
response = llm.invoke(user_prompt)
print(response.content)


1. search type: similarity
'AII-in 코딩 공모전' 수상작 요약:

1. **대상: Lexi Note**
   - **서비스 내용:** 언어공부 필기 웹 서비스
   - **제작자:** 다나와(김다애, 박나경)
   - **기술 스택:** 정보 없음

2. **우수상: 에코 클래스룸**
   - **서비스 내용:** 수업 실시간 소통 서비스
   - **제작자:** This is 스파게티!!!(박지성, 김서원, 박범수)
   - **기술 스택:** Flutter, Socket.IO, Expo CLI, Axios, TanStack Query, Spring Boot, Spring Security, JWT, MySQL, Spring WebSocket, AWS

3. **우수상: 우리집 히어로즈**
   - **서비스 내용:** 벌레 퇴치 영웅 매칭 서비스
   - **제작자:** 인트(배정연, 한지수)
   - **기술 스택:** React, Tesseract.js, React-Quill, HTML, CSS, JavaScript, Java, Spring Boot, MariaDB

4. **입선: Crewing**
   - **서비스 내용:** 연합동아리 정보 플랫폼
   - **제작자:** 동학대학운동(김민아, 임경진, 신은혜, 고수)
   - **기술 스택:** Spring Boot, Redis, MySQL, SwiftUI Framework, OAuth 2.0

5. **입선: 학교생활 매니저**
   - **서비스 내용:** 학교생활 관리 서비스
   - **제작자:** 아이칼F4(조민제, 이민기, 강건, 박근우)
   - **기술 스택:** 정보 없음

이번 공모전은 대학생들이 캠퍼스에서 경험한 문제를 해결하기 위한 다양한 아이디어와 혁신적인 프로젝트가 발표되었습니다. 각 수상작들은 실용적이고 참신한 해결책을 제시하여 참가자들의 열정을 엿볼 수 있는 기회를 제공했습니다.



2. search type: mmr
### AII-in 코딩

Self-Query 리트리버가 가장 효과적인 결과를 보여주었습니다. 이 방식은 메타데이터 필드(상장 종류, 공모전 이름, 수상자 등)를 명확히 정의하여 LLM이 사용자 질문을 구조화된 쿼리로 변환할 수 있게 했습니다. 그 결과, 다른 리트리버들이 놓친 "BLOTIE" 입선작까지 포함한 더 포괄적인 정보를 제공했고, 프론트엔드/백엔드 기술 스택 구분, 서비스 기능 설명, 문제 해결 방식 등 더 체계적인 정보를 추출할 수 있었습니다. 특히 특정 속성에 대한 정보를 정확하게 추출해야 하는 상황에서 Self-Query 리트리버의 강점이 두드러졌으며, 이는 복잡한 정보를 구조화하여 검색하는 데 효과적인 접근법임을 보여줍니다.